In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [6]:
# loading the text data
txt_loader = TextLoader('aitrading.txt')
text_data = txt_loader.load()
text_data


[Document(metadata={'source': 'aitrading.txt'}, page_content='Artificial intelligence (AI) has revolutionized various industries, and stock trading is no exception. The integration of AI in stock trading has led to the development of sophisticated algorithms capable of analyzing vast amounts of data at unprecedented speeds. Traders and investors now rely on AI to make informed decisions based on real-time market trends, historical data, and predictive analytics. The accuracy and efficiency of AI-driven trading systems have outpaced traditional methods, enabling the identification of profitable opportunities with minimal human intervention.\n\nThese AI systems use machine learning models, such as neural networks and decision trees, to process and interpret complex datasets. By continuously learning from past performance and adapting to new information, AI can predict price movements, assess risks, and optimize trading strategies. Natural language processing (NLP) is another crucial comp

In [9]:
# splitting the data using character text splitter
splitter = CharacterTextSplitter(separator='\n\n',chunk_size=300,chunk_overlap=50)
splitted_docs = splitter.split_documents(text_data)
splitted_docs

Created a chunk of size 585, which is longer than the specified 300
Created a chunk of size 679, which is longer than the specified 300
Created a chunk of size 437, which is longer than the specified 300


[Document(metadata={'source': 'aitrading.txt'}, page_content='Artificial intelligence (AI) has revolutionized various industries, and stock trading is no exception. The integration of AI in stock trading has led to the development of sophisticated algorithms capable of analyzing vast amounts of data at unprecedented speeds. Traders and investors now rely on AI to make informed decisions based on real-time market trends, historical data, and predictive analytics. The accuracy and efficiency of AI-driven trading systems have outpaced traditional methods, enabling the identification of profitable opportunities with minimal human intervention.'),
 Document(metadata={'source': 'aitrading.txt'}, page_content='These AI systems use machine learning models, such as neural networks and decision trees, to process and interpret complex datasets. By continuously learning from past performance and adapting to new information, AI can predict price movements, assess risks, and optimize trading strateg

In [10]:
## text embeddings
embedding  = OllamaEmbeddings(model='gemma:2b')
query_result = embedding.embed_documents(splitted_docs)
query_result

[[0.5949041843414307,
  -3.2862205505371094,
  -1.2556078433990479,
  0.8136119842529297,
  0.6157264113426208,
  0.625796377658844,
  1.126420259475708,
  -0.46049973368644714,
  1.406661033630371,
  -1.6739542484283447,
  -0.5169995427131653,
  -0.3509231209754944,
  -0.13717174530029297,
  -0.09465442597866058,
  0.5837700366973877,
  0.8564648628234863,
  1.5869383811950684,
  0.950870156288147,
  0.4484906792640686,
  0.40704435110092163,
  -1.1683787107467651,
  0.42053475975990295,
  1.5349867343902588,
  1.1199098825454712,
  1.3258733749389648,
  0.08659815043210983,
  -0.3362872004508972,
  0.3370901048183441,
  -0.7835769653320312,
  -2.057569980621338,
  -0.023473942652344704,
  -1.2528434991836548,
  0.7210513949394226,
  0.3020989000797272,
  0.755160927772522,
  0.2500036358833313,
  -2.2839953899383545,
  -0.8985483646392822,
  0.0999465137720108,
  -1.3755090236663818,
  -0.41625043749809265,
  0.0815550684928894,
  -0.8441302180290222,
  -1.0228112936019897,
  2.75667

In [17]:
print("no of docs:",len(query_result))
print("shape of each doc:",len(query_result[0]))


no of docs: 4
shape of each doc: 2048


In [22]:
# using faiss to store the vectors
db = FAISS.from_documents(splitted_docs,embedding)
db

In [23]:
# vectors are stored in db and to query something
query = "does ths adaption of ai in stocks are with challenges?"
queried_docs = db.similarity_search(query)
queried_docs
# we get the similar docs from db

[Document(metadata={'source': 'aitrading.txt'}, page_content='Despite these challenges, the benefits of AI in stock trading are undeniable. As technology continues to advance, AI-driven trading systems are expected to become even more sophisticated, further transforming the financial markets. Investors who embrace AI are likely to gain a significant advantage, while those who resist may find themselves struggling to keep up with the rapid pace of change in the industry.'),
 Document(metadata={'source': 'aitrading.txt'}, page_content='Artificial intelligence (AI) has revolutionized various industries, and stock trading is no exception. The integration of AI in stock trading has led to the development of sophisticated algorithms capable of analyzing vast amounts of data at unprecedented speeds. Traders and investors now rely on AI to make informed decisions based on real-time market trends, historical data, and predictive analytics. The accuracy and efficiency of AI-driven trading system

### Using db as a Retriever

In [24]:
# we can convert db as a a retiever, this allows us to use in other langchain methods which work with retrievers
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs

[Document(metadata={'source': 'aitrading.txt'}, page_content='Despite these challenges, the benefits of AI in stock trading are undeniable. As technology continues to advance, AI-driven trading systems are expected to become even more sophisticated, further transforming the financial markets. Investors who embrace AI are likely to gain a significant advantage, while those who resist may find themselves struggling to keep up with the rapid pace of change in the industry.'),
 Document(metadata={'source': 'aitrading.txt'}, page_content='Artificial intelligence (AI) has revolutionized various industries, and stock trading is no exception. The integration of AI in stock trading has led to the development of sophisticated algorithms capable of analyzing vast amounts of data at unprecedented speeds. Traders and investors now rely on AI to make informed decisions based on real-time market trends, historical data, and predictive analytics. The accuracy and efficiency of AI-driven trading system

In [25]:
# we have some specific methods in FASSI, we can get similarity_search_with_score which is distance score b/w query and docs
# low score is better
docs_with_score = db.similarity_search_with_score(query)
docs_with_score

[(Document(metadata={'source': 'aitrading.txt'}, page_content='Despite these challenges, the benefits of AI in stock trading are undeniable. As technology continues to advance, AI-driven trading systems are expected to become even more sophisticated, further transforming the financial markets. Investors who embrace AI are likely to gain a significant advantage, while those who resist may find themselves struggling to keep up with the rapid pace of change in the industry.'),
  2157.9673),
 (Document(metadata={'source': 'aitrading.txt'}, page_content='Artificial intelligence (AI) has revolutionized various industries, and stock trading is no exception. The integration of AI in stock trading has led to the development of sophisticated algorithms capable of analyzing vast amounts of data at unprecedented speeds. Traders and investors now rely on AI to make informed decisions based on real-time market trends, historical data, and predictive analytics. The accuracy and efficiency of AI-drive

In [26]:
# to save the file to local
db.save_local('faiss_index')

In [27]:
# to load the db 
new_db  = FAISS.load_local("faiss_index",allow_dangerous_deserialization=True,embeddings=embedding)
new_db.similarity_search(query)

[Document(metadata={'source': 'aitrading.txt'}, page_content='Despite these challenges, the benefits of AI in stock trading are undeniable. As technology continues to advance, AI-driven trading systems are expected to become even more sophisticated, further transforming the financial markets. Investors who embrace AI are likely to gain a significant advantage, while those who resist may find themselves struggling to keep up with the rapid pace of change in the industry.'),
 Document(metadata={'source': 'aitrading.txt'}, page_content='Artificial intelligence (AI) has revolutionized various industries, and stock trading is no exception. The integration of AI in stock trading has led to the development of sophisticated algorithms capable of analyzing vast amounts of data at unprecedented speeds. Traders and investors now rely on AI to make informed decisions based on real-time market trends, historical data, and predictive analytics. The accuracy and efficiency of AI-driven trading system